<a href="https://colab.research.google.com/github/mehdi780311/Innovation-Evaluation/blob/main/doc-based-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import json
import base64
import io
import time
import requests

# --- تنظیمات اولیه ---

# استفاده از متغیر سراسری API Key در محیط Canvas یا تنظیم آن به عنوان یک رشته خالی
API_KEY = "" # در صورت نیاز، کلید API خود را اینجا قرار دهید یا از متغیرهای محیطی استفاده کنید
API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-05-20:generateContent"

# پیکربندی LLM برای خروجی ساختاریافته (JSON Schema)
LLM_SCHEMA = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "عنوان_پایان_نامه": {"type": "STRING", "description": "عنوان تشخیص داده شده."},
            "چکیده_پایان_نامه": {"type": "STRING", "description": "چکیده کامل تشخیص داده شده."},
            "امتیاز_شاخص_۱": {"type": "INTEGER", "description": "امتیاز (0 تا 2): آیا در حوزه‌های با بیشترین ارجاع به پتنت قرار دارد؟"},
            "امتیاز_شاخص_۲": {"type": "INTEGER", "description": "امتیاز (0 تا 3): آیا به تکنولوژی، مدل، محصول، الگوریتم یا متدولوژی جدید اشاره می‌کند؟"},
            "امتیاز_شاخص_۳": {"type": "INTEGER", "description": "امتیاز (0 تا 2): آیا به یه نیاز یا مسئله کاربردی خاصی اشاره می‌کند؟"},
            "امتیاز_شاخص_۴": {"type": "INTEGER", "description": "امتیاز (0 تا 2): آیا قابلیت توسعه به محصول، نرم‌افزار یا راهکار را دارد؟"},
            "امتیاز_شاخص_۵": {"type": "INTEGER", "description": "امتیاز (0 تا 1): آیا نشان دهنده همکاری با نهاد صنعتی یا سازمانی است؟"},
            "امتیاز_کل": {"type": "INTEGER", "description": "جمع امتیازات 1 تا 5 (حداکثر 10)."},
            "پتانسیل": {"type": "STRING", "description": "پتانسیل (ضعیف: کمتر از 5، متوسط: 5 تا 7، بالا: 8 تا 10) بر اساس امتیاز کل."}
        },
        "required": ["عنوان_پایان_نامه", "چکیده_پایان_نامه", "امتیاز_شاخص_۱", "امتیاز_شاخص_۲", "امتیاز_شاخص_۳", "امتیاز_شاخص_۴", "امتیاز_شاخص_۵", "امتیاز_کل", "پتانسیل"]
    }
}

# دستور سیستم برای هدایت عملکرد LLM
SYSTEM_PROMPT = (
    "شما یک کارشناس ارشد تجاری‌سازی فناوری و تحلیلگر پتنت هستید. "
    "وظیفه شما تحلیل متنی است که حاوی چندین عنوان و چکیده پایان‌نامه است. "
    "باید هر عنوان و چکیده را به صورت جداگانه بر اساس شاخص‌های زیر ارزیابی کرده و پاسخ را دقیقاً در قالب JSON مشخص شده برگردانید. "
    "امتیازدهی باید کاملاً مطابق با مقادیر مجاز برای هر شاخص باشد و پتانسیل نهایی را بر اساس مجموع امتیاز کل محاسبه کنید. "
    "لطفاً فقط خروجی JSON را تولید کنید و از افزودن هرگونه متن اضافی در خارج از بدنه JSON خودداری نمایید."
)

# --- توابع کمکی ---

def extract_text_from_file(uploaded_file):
    """
    تابع شبیه‌سازی شده برای استخراج متن از فایل.
    در یک محیط عملیاتی، این تابع باید از کتابخانه‌هایی مانند:
    - PyPDF2/pdfplumber برای PDF
    - python-docx برای DOCX
    استفاده کند.
    """
    st.info("لطفاً توجه داشته باشید: در این محیط شبیه‌سازی، استخراج دقیق متن از PDF/Word ممکن است با محدودیت همراه باشد. در محیط لوکال، شما باید کتابخانه‌های مناسب (مانند pdfplumber یا python-docx) را نصب و استفاده کنید.")

    # شبیه‌سازی استخراج برای نمایش عملکرد LLM
    mock_text = """
    عنوان: طراحی و ساخت سنسور پوشیدنی برای نظارت بر گلوکز غیرتهاجمی
    چکیده: این پایان‌نامه به طراحی یک حسگر زیستی پوشیدنی جدید می‌پردازد که می‌تواند غلظت گلوکز را از طریق عرق اندازه‌گیری کند، بدون نیاز به نمونه‌گیری خون. این حسگر از نانوذرات طلا اصلاح‌شده (AuNPs) برای افزایش حساسیت استفاده می‌کند و نتایج آن در مقایسه با روش‌های تهاجمی استاندارد اعتبارسنجی شده است. این تحقیق با همکاری یک شرکت تولید تجهیزات پزشکی (MediTech Co.) انجام شده است تا نمونه اولیه برای تجاری‌سازی آماده شود.
    ---
    عنوان: بررسی تأثیر رنگ فونت بر حافظه در کودکان
    چکیده: این مطالعه تأثیر متغیرهای بصری ساده، به ویژه رنگ فونت، بر قابلیت یادآوری واژگان در یک گروه از کودکان پیش‌دبستانی را بررسی می‌کند. نتایج نشان داد که رنگ‌های گرم تأثیر مثبتی بر یادآوری کوتاه‌مدت دارند. این تحقیق صرفاً یک مطالعه آکادمیک در حوزه روانشناسی شناختی است و هدف توسعه محصول خاصی را دنبال نمی‌کند.
    """
    return mock_text

@st.cache_data
def convert_df_to_excel(df):
    """تبدیل DataFrame به فایل اکسل (BytesIO)"""
    output = io.BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='تحلیل پتانسیل')
    processed_data = output.getvalue()
    return processed_data

def call_gemini_api(prompt_text):
    """
    فراخوانی API مدل Gemini با خروجی ساختاریافته (JSON)
    """
    if not API_KEY and not st.session_state.get('initial_auth_complete'):
        st.error("کلید API برای مدل‌های Gemini فراهم نشده است. لطفا کلید را وارد کنید یا مطمئن شوید که در محیط Canvas تنظیم شده است.")
        return None

    # بدنه درخواست (Payload)
    payload = {
        "contents": [{"parts": [{"text": prompt_text}]}],
        "systemInstruction": {"parts": [{"text": SYSTEM_PROMPT}]},
        "generationConfig": {
            "responseMimeType": "application/json",
            "responseSchema": LLM_SCHEMA
        }
    }

    headers = {'Content-Type': 'application/json'}

    # اجرای Exponential Backoff برای مدیریت خطاهای API
    max_retries = 5
    for attempt in range(max_retries):
        try:
            response = requests.post(f"{API_URL}?key={API_KEY}", headers=headers, json=payload, timeout=60)
            response.raise_for_status() # برای برانگیختن خطا برای کدهای وضعیت بد (4xx یا 5xx)

            result = response.json()

            # استخراج محتوای JSON
            if result.get('candidates') and result['candidates'][0].get('content'):
                json_string = result['candidates'][0]['content']['parts'][0]['text']

                # رفع مشکل احتمالی JSON (اگر مدل رشته‌های اضافی قبل/بعد از JSON تولید کند)
                try:
                    return json.loads(json_string)
                except json.JSONDecodeError:
                    st.warning("مدل خروجی غیر استاندارد JSON برگرداند. تلاش برای پاکسازی...")
                    # تلاش برای یافتن JSON در داخل رشته
                    start = json_string.find('[')
                    end = json_string.rfind(']')
                    if start != -1 and end != -1:
                        return json.loads(json_string[start:end+1])

            st.error("پاسخ مدل ساختار مورد انتظار را نداشت.")
            return None

        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1 and (response.status_code >= 500 or response.status_code == 429):
                wait_time = 2 ** attempt
                st.warning(f"خطای API ({e}). تلاش مجدد در {wait_time} ثانیه...")
                time.sleep(wait_time)
            else:
                st.error(f"خطا در فراخوانی API: {e}")
                return None
    return None

# --- رابط کاربری Streamlit ---

st.set_page_config(
    page_title="تحلیل پتانسیل تجاری پایان‌نامه‌ها",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.markdown(
    """
    <style>
    .stApp {
        background-color: #f0f2f6;
    }
    .main .block-container {
        padding-top: 2rem;
        padding-bottom: 2rem;
    }
    h1 {
        color: #1f3f66;
        text-align: center;
    }
    .stDownloadButton button {
        background-color: #1f3f66;
        color: white;
        border-radius: 8px;
        padding: 10px 20px;
        font-weight: bold;
    }
    .stDownloadButton button:hover {
        background-color: #295083;
    }
    </style>
    """,
    unsafe_allow_html=True
)

st.title("💡 تحلیل پتانسیل تجاری پایان‌نامه‌ها با هوش مصنوعی")

st.markdown("با آپلود فایل حاوی چکیده‌های پایان‌نامه (PDF یا Word)، پتانسیل تجاری هر پژوهش را بر اساس شاخص‌های پتنت، نوآوری، کاربرد و قابلیت توسعه ارزیابی کنید.")
st.markdown("---")

# 1. بارگذاری فایل
uploaded_file = st.file_uploader(
    "فایل چکیده‌های پایان‌نامه (PDF/Word) را آپلود کنید:",
    type=["pdf", "docx"],
    accept_multiple_files=False,
    help="در محیط لوکال خود می‌توانید فایل‌های واقعی را تحلیل کنید. در این دمو، از یک متن شبیه‌سازی شده استفاده می‌شود."
)

# 2. منطق دکمه تحلیل
if uploaded_file is not None:
    # شبیه‌سازی استخراج متن
    raw_text = extract_text_from_file(uploaded_file)
    st.session_state['extracted_text'] = raw_text
    st.text_area("متن استخراج شده (شبیه‌سازی شده):", raw_text, height=200, disabled=True)

    if st.button("تحلیل پتانسیل با هوش مصنوعی", key="analyze_button", type="primary"):
        with st.spinner('در حال ارسال داده‌ها به مدل هوش مصنوعی و تحلیل...'):
            # فراخوانی LLM
            llm_result_data = call_gemini_api(
                f"متن زیر شامل چندین عنوان و چکیده پایان‌نامه است. آن‌ها را شناسایی و طبق شاخص‌های درخواستی تحلیل کنید:\n\n{raw_text}"
            )

            if llm_result_data:
                try:
                    df_results = pd.DataFrame(llm_result_data)

                    # اطمینان از اعمال فیلد "پتانسیل" بر اساس قوانین
                    def determine_potential(score):
                        if score < 5:
                            return "پتانسیل ضعیف (کمتر از 5)"
                        elif 5 <= score <= 7:
                            return "پتانسیل متوسط (5 تا 7)"
                        else:
                            return "پتانسیل بالا (8 تا 10)"

                    df_results['پتانسیل'] = df_results['امتیاز_کل'].apply(determine_potential)

                    st.session_state['analysis_df'] = df_results
                    st.success("تحلیل با موفقیت انجام شد!")
                except Exception as e:
                    st.error(f"خطا در پردازش خروجی مدل: {e}")
                    st.json(llm_result_data)
            else:
                st.error("عملیات تحلیل ناموفق بود یا خروجی ساختاریافته مورد انتظار برگردانده نشد.")
else:
    # پاک کردن حالت تحلیل در صورت عدم وجود فایل
    if 'analysis_df' in st.session_state:
        del st.session_state['analysis_df']
    if 'extracted_text' in st.session_state:
        del st.session_state['extracted_text']


# 3. نمایش نتایج و دکمه دانلود
if 'analysis_df' in st.session_state:
    st.markdown("---")
    st.subheader("جدول نتایج تحلیل پتانسیل")

    df_display = st.session_state['analysis_df']

    # سفارشی‌سازی نمایش DataFrame
    df_styled = df_display.style.background_gradient(
        cmap='YlOrRd',
        subset=['امتیاز_کل']
    ).set_properties(
        **{'font-size': '12pt', 'text-align': 'right'}
    )

    st.dataframe(df_styled, use_container_width=True)

    # دکمه دانلود فایل اکسل
    excel_data = convert_df_to_excel(df_display)
    st.download_button(
        label="⬇️ دانلود فایل اکسل نتایج",
        data=excel_data,
        file_name="تحلیل_پتانسیل_پایان_نامه‌ها.xlsx",
        mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
        key='download_excel'
    )

# 4. دکمه بازنشانی
def reset_app():
    """بازنشانی کامل وضعیت برنامه"""
    for key in ['analysis_df', 'extracted_text']:
        if key in st.session_state:
            del st.session_state[key]
    st.rerun()

st.markdown("---")
if st.button("🔄 بازنشانی برنامه", key="reset_button"):
    reset_app()

st.markdown(
    """
    <div style="font-size: small; color: gray; margin-top: 30px;">
    **توضیحات مهم در مورد شاخص‌ها:**<br>
    - **شاخص ۱ (0-2):** بیشترین ارجاع به پتنت (تمرکز بر حوزه‌های داغ و ثبت اختراع شده).<br>
    - **شاخص ۲ (0-3):** نوآوری (تکنولوژی، مدل، محصول، الگوریتم یا متدولوژی جدید).<br>
    - **شاخص ۳ (0-2):** کاربردی بودن (اشاره به نیاز یا مسئله کاربردی خاص).<br>
    - **شاخص ۴ (0-2):** قابلیت توسعه (قابلیت تبدیل به محصول/نرم‌افزار/راهکار).<br>
    - **شاخص ۵ (0-1):** همکاری صنعتی (نشان دهنده ارتباط با سازمان/صنعت).<br>
    - **پتانسیل:** ضعیف: <5، متوسط: 5-7، بالا: 8-10.
    </div>
    """,
    unsafe_allow_html=True
)